### Import packages

In [1]:
#required packages
import requests
from bs4 import BeautifulSoup as bs
import numpy as np
import pandas as pd
import os
import geopandas as gpd
import rasterio
import rasterstats
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime
import scipy.sparse as sparse
import patoolib
from osgeo import gdal

## Intro

This script downloads daily rainfall rasters from CHIRPS (https://www.chc.ucsb.edu/data/chirps), unzips and clips the rasters to the desired study area and the extract point timeseries for multiple stations

### First!! 

Create folders to save the downloaded data. 

### Download the raw tiff files 

The files are available from this url:https://data.chc.ucsb.edu/products/CHIRPS-2.0/africa_daily/tifs/p05/. Each folder contains daily data for each year. Each TIFF file contains data for the entire African continent.
Recommended: First explore the data to understand the format and contents. You can manually download one file to compare it with the output of the code

Summary of the code:
First generate list of days in a month and months in a years
The format of the data is 'chirps-v2.0.1981.01.01.tif.gz' so we'll need to generate a list of numbers where the number between 1 and 9 are preceded by a zero i.e 01,02,03, etc. We'll call these days two_sf_days i.e 2 significant figures days
we will also follow the same procedure for months. Iterate throught url and extract daily zipped tiff files

Dont forget to delete downloaded files which do not exist e.g the code assumes all months have 31 days and will for example download data for 31st February which does not exists. You can identify these files by their size. They will mostly have a 1KB size. Delete these files before unzipping the TIFFs

In [49]:
os.chdir(r'H:\CHIRPS_data\afr_daily') #path to zipped tiff files

#generate list of days in 2 significant figures
two_sf_days=[] #empty list to store the days
days=list(np.arange(1,32)) #generate 1 to 31 days
for day in days:
          day_dd=("%02d" % (day,))
          two_sf_days.append(day_dd) 

#generate list of days in 2 significant figures
two_sf_months=[] #empty list to store months
months=list(np.arange(1,13)) #generates 12 months
for mon in months:
          month_mm=("%02d" % (mon,))
          two_sf_months.append(month_mm)

#generate list of years from 1981-2022 . np.arange is end exclusive so 2023 will not be included
years=list(np.arange(1981,2023))

#iterate through the folders and extract the daily data
for yr in years:
    for month in two_sf_months:
        for day in two_sf_days:
            url='https://data.chc.ucsb.edu/products/CHIRPS-2.0/africa_daily/tifs/p05/'+ str(yr) + '/' + 'chirps-v2.0.'+str(yr)+ '.'+ str(month) + '.'+ str(day) + '.tif.gz'
            r=requests.get(url, stream=True)      
            open('chirps_'+str(yr)+'_' + str(month)+'_'+str(day)+'.gz', 'wb').write(r.content)

### Unzip the files

In [ ]:
#iterate through the zipped files and unzip and save to unzipped folder. Make sure you change the file path
for zipfile in os.listdir(r'H:\CHIRPS_data\afr_daily'):
    #print(zipfiles)
    if zipfile[-3:]=='.gz':
        patoolib.extract_archive(zipfile, outdir=r'H:\CHIRPS_data\afr_daily')
        
#optional
#add file extensions
#for unzipped_file in os.listdir(r'H:\CHIRPS_data\unzipped_files'):
    #os.chdir(r'H:\CHIRPS_data\unzipped_files')
    #os.rename(unzipped_file, unzipped_file)# +'.tif')

### Clip the raster to catchment area

This helps to reduce memory committed to processing the files.
If the catchment is small, clip the raster files using a bigger shapefile e.g the boundaries of a country

In [ ]:
define paths to zipped files and the output path where unzipped files will be saved
input_path=r'H:\CHIRPS_data\afr_daily' #change paths to folders accordingly
output_path=r'H:\CHIRPS_data\EA_rasters'
        
#define path to shapefile
shapefile=r"H:\CHIRPS_data\EA_Clip.shp"

raster_list = [raster for raster in os.listdir(input_path) if raster[-4:]=='.tif']

#clip all the selected raster files with the Warp option from GDAL
for raster in raster_list:
    print(output_path)
    options = gdal.WarpOptions(cutlineDSName=shapefile,cropToCutline=True)
    outBand = gdal.Warp(srcDSOrSrcDSTab=os.path.join(input_path, raster),
                        destNameOrDestDS=os.path.join(output_path, raster[:-4]+raster[-4:]),
                        options=options)
    outraster= None

### Extract station data

For point data extraction, create a shapefile containing the name, lat and lon of your stations. 
The lat/lon should be in decimal degree. This video instructs how to import point from data from Excel and convert to shapefile https://www.youtube.com/watch?v=Xgi-UdyDi1k

In [ ]:
# Create an empty pandas dataframe called 'table'
table = pd.DataFrame(index = np.arange(0,1))

#First create a shapefile of points stations 
#Read the points shapefile using GeoPandas 
stations = gpd.read_file(r'H:\CHIRPS_data\unzipped_files\chirps_stations.shp')
stations['lon'] = stations['geometry'].x
stations['lat'] = stations['geometry'].y

Matrix = pd.DataFrame()

# Iterate through the rasters and save the data as individual arrays to a Matrix 
for files in os.listdir(r'H:\CHIRPS_data\EA_rasters''):
    if files[-4: ] == '.tif':
        dataset = rasterio.open(r'H:\CHIRPS_data\EA_rasters'+'\\'+files)
        data_array = dataset.read(1)
        data_array_sparse = sparse.coo_matrix(data_array, shape = (1600,1500))
        data = files[ :-4]
        Matrix[data] = data_array_sparse.toarray().tolist()
        print('Processing is done for the raster: '+ files[:-4])

# Iterate through the stations and get the corresponding row and column for the related x, y coordinates    
for index, row in stations.iterrows():
        station_name = str(row['station_na'])#station_na refers to the station name in the stations shapefile
        lon = float(row['lon'])
        lat = float(row['lat'])
        x,y = (lon, lat)
        row, col = dataset.index(x, y)
        print('Processing: '+ station_name)
        
        # Pick the rainfall value from each stored raster array and record it into the previously created 'table'
        for records_date in Matrix.columns.tolist():
            a = Matrix[records_date]
            rf_value = a.loc[int(row)][int(col)]
            table[records_date] = rf_value
            transpose_mat = table.T
            transpose_mat.rename(columns = {0: 'Rainfall(mm)'}, inplace = True)
        
            transpose_mat.to_csv(r'H:\CHIRPS_data\EA_rasters'+'\\'+station_name+'.txt') #provide path to station data